# Submission details
*   **Group**: 45 
*   **Name**: Aditya Verma 12948511, Elena Varelas 12567487, Jenny Nguyen 12948461
*   **Task**: Vehicle Number plate Object detection
*   **Subject**: 42028 Deep Learning and Convolutional Neural Network - Autumn 2020
*   **DataSet**: Isreal Dataset
*   **CNN Used**: Multiple




Donwloaded the required packages 

In [ ]:
%tensorflow_version 1.x
!pip install numpy==1.17.4

TensorFlow 1.x selected.


In [ ]:
import re
import numpy as np

import os
from glob import glob
import shutil

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import tensorflow.compat.v1 as tf 

Mounted the drive so we could easily access our files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/assign3

/content/gdrive/My Drive/assign3


#1 Download Dataset

In [ ]:
!pip install requests; pip install pillow

Downloaded the dataset into the appropriate folders

In [ ]:
import os
image_download_folder = os.path.join(os.getcwd(),'Images')
pascal_voc_out_folder = os.path.join(os.getcwd(),'Pascal_Voc')
filepath = 'Israeli_license_plates.json'

In [ ]:
!python dataturks_to_PascalVOC.py $filepath ./Images ./Pascal_Voc/

INFO:root:10 items done ...
INFO:root:20 items done ...
INFO:root:30 items done ...
INFO:root:40 items done ...
INFO:root:50 items done ...
INFO:root:60 items done ...
INFO:root:70 items done ...
INFO:root:80 items done ...
INFO:root:90 items done ...
INFO:root:100 items done ...
INFO:root:110 items done ...
INFO:root:120 items done ...
INFO:root:130 items done ...
INFO:root:140 items done ...
INFO:root:150 items done ...
INFO:root:160 items done ...
INFO:root:170 items done ...
INFO:root:180 items done ...
INFO:root:190 items done ...
INFO:root:200 items done ...
INFO:root:210 items done ...
INFO:root:220 items done ...
INFO:root:230 items done ...
INFO:root:240 items done ...
INFO:root:250 items done ...
INFO:root:260 items done ...
INFO:root:270 items done ...
INFO:root:280 items done ...
INFO:root:290 items done ...
INFO:root:300 items done ...
INFO:root:310 items done ...
INFO:root:320 items done ...
INFO:root:330 items done ...
INFO:root:340 items done ...
INFO:root:350 items don

#2 Model

Set up training parameters for the model and selected the model type (changed model type for each experiment)

In [ ]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/Tony607/object_detection_demo'

# Number of training steps.
num_steps = 5000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    }
}

# Pick the model you want to use

selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
%cd /content/gdrive/My Drive/assign3

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content/gdrive/My Drive/assign3
fatal: destination path 'object_detection_demo' already exists and is not an empty directory.
/content/gdrive/My Drive/assign3/object_detection_demo
Already up to date.


DOwnloaded pre-trained models 

In [ ]:
%cd /content

#!git clone --quiet https://github.com/tensorflow/models.git

!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models.git


!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
Cloning into 'models'...
remote: Enumerating objects: 2927, done.
remote: Counting objects: 100% (2927/2927), done.
remote: Compressing objects: 100% (2449/2449), done.
remote: Total 2927 (delta 509), reused 2035 (delta 403), pack-reused 0
Receiving objects: 100% (2927/2927), 369.04 MiB | 39.80 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Checking out files: 100% (2768/2768), done.
Selecting previously unselected package python-bs4.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-si

#3 Preparing tfrecord files

##3.x MANUALLY move Images and corresponding xml
I have manually moved the images and xml files into seperate folders.
The split format is roughly 90% Training to 10% Test(51 Images).
Images and xml ranging from 00000 to 00351 have been moved to /images/train. And the rest have been moved to /images/test/.

Overwrites existing image folder with the blood images

In [ ]:
%cd /content/gdrive/My Drive/assign3

/content/gdrive/My Drive/assign3


In [ ]:
#Moves the images into the demo github repo
!mv images object_detection_demo/data/

mv: cannot stat 'images': No such file or directory


##3.1 Creating tf records and label maps

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%cd {repo_dir_path}

# Using the xml to cxs python file to convert the annotations in the 
# training folder to csv files in preperation for training
# It will also be creating a label map
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# to generate the csv file for the test images
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

/content/gdrive/My Drive/assign3/object_detection_demo
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


In [ ]:
%cd {repo_dir_path}
# tf records are required for the labels so that the model can train. 
# The python file generate tfrecords from the tf model dir is being used
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# tf records are required for the labels so that the model can train. 
# The python file generate tfrecords from the tf model dir is being used
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

/content/gdrive/My Drive/assign3/object_detection_demo


W0616 04:50:52.109442 140173062334336 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0616 04:50:52.845963 140173062334336 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Traceback (most recent call last):
  File "generate_tfrecord.py", line 134, in <module>
    tf.app.run()
  File "/tensorflow-1.15.2/python3.6/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "generate_tfrecord.py", line 125, in main
    tf_

##3.2 Initializing directories 

In [ ]:
test_record_fname = '/content/gdrive/My Drive/assign3//object_detection_demo/data/annotations/test.record'
train_record_fname = '/content/gdrive/My Drive/assign3/object_detection_demo/data/annotations/train.record'
label_map_pbtxt_fname = '/content/gdrive/My Drive/assign3/object_detection_demo/data/annotations/label_map.pbtxt'

#4 Training

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 71 root   root 4.0K Jun 12 10:49 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 2
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        tru

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

##tensorboard

Creation of tensorboard to easily display our results and training progress in real time

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-06-12 10:49:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.223.98.104, 52.22.23.115, 52.45.174.78, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.223.98.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.5’

ngrok-stable-linux- 100%[===================>]  13.13M  5.98MB/s    in 2.2s    

2020-06-12 10:49:52 (5.98 MB/s) - ‘ngrok-stable-linux-amd64.zip.5’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')


In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://67cab70a4df3.ngrok.io


##Training

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}